In [1]:
# importações
from utils import *
from utils_backtest import *
from models import *
from config import *

# 1. Aplicabilidade do Modelo 2 no Mercado Real

In [3]:
# Lista de ações
stock_list = ['PETR4', 'VALE3', 'BBDC4','ITUB4']

# Carrega os dados de todas as ações e calcula os indicadores e salva em um dicionario
all_stock_data = load_stocks_data(stock_list)

Carregando dados para: PETR4
Carregando dados para: VALE3
Carregando dados para: BBDC4
Carregando dados para: ITUB4


## 2. Processar o Treino do Modelo para Salvar os artefatos

In [7]:
# Processando o Modelo 2 para Salvar os Artefatos (modelo e scalers)

name_model = model_2

# Usando a Combinação 4: ("Stock Data + IT (IFR + MMS + MME)", stock_data + ifr + mms + mme),
result = process_stocks_and_save_metrics(all_stock_data, 4, name_model, save_artifacts=True) # Salva os artefatos

model_2: Processando PETR4 | Combinação 4...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 0.0518 - mae: 0.1621 - mape: 6856.5435 - mse: 0.0518 - val_loss: 5.9772e-04 - val_mae: 0.0173 - val_mape: 4.2749 - val_mse: 5.9772e-04
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 4.3298e-04 - mae: 0.0158 - mape: 32102.9062 - mse: 4.3298e-04 - val_loss: 4.1488e-04 - val_mae: 0.0142 - val_mape: 3.4324 - val_mse: 4.1488e-04
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.6254e-04 - mae: 0.0118 - mape: 5081.5391 - mse: 2.6254e-04 - val_loss: 3.6067e-04 - val_mae: 0.0128 - val_mape: 3.0917 - val_mse: 3.6067e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 2.2372e-04 - mae: 0.0109 - mape: 6884.3960 - mse: 2.2372e-04 - val_loss: 3.7522e-04 - val_mae: 0.0134 - val_mape: 3.2017 - val_mse: 3.7522e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.3137e-04 - mae: 0.0110 - mape: 3719.1633 - mse: 2.3137e-04 - val_loss: 3.9915e-04 - val_mae: 

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
model_2: Processando VALE3 | Combinação 4...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 0.0812 - mae: 0.2060 - mape: 31349.8555 - mse: 0.0812 - val_loss: 0.0192 - val_mae: 0.1043 - val_mape: 8.6617 - val_mse: 0.0192
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 6.5443e-04 - mae: 0.0198 - mape: 45249.4297 - mse: 6.5443e-04 - val_loss: 0.0027 - val_mae: 0.0394 - val_mape: 3.7017 - val_mse: 0.0027
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 4.1602e-04 - mae: 0.0153 - mape: 13635.9580 - mse: 4.1602e-04 - val_loss: 0.0015 - val_mae: 0.0291 - val_mape: 2.9477 - val_mse: 0.0015
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 3.5246e-04 - mae: 0.0140 - mape: 4044.5918 - mse: 3.5246e-04 - val_loss: 0.0019 - val_mae: 0.0322 - val_mape: 3.0203 - val_mse: 0.0019
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 3.4942e-04 - mae: 0.0141 - mape: 32968.6289 - mse: 3.4942e-04 - val_loss: 0.0038 - va

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
model_2: Processando BBDC4 | Combinação 4...
Epoch 1/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 0.0563 - mae: 0.1599 - mape: 76331.9766 - mse: 0.0563 - val_loss: 0.0012 - val_mae: 0.0254 - val_mape: 4.8897 - val_mse: 0.0012
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 7.8550e-04 - mae: 0.0202 - mape: 9247.3965 - mse: 7.8550e-04 - val_loss: 7.3394e-04 - val_mae: 0.0191 - val_mape: 3.7647 - val_mse: 7.3394e-04
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 4.8210e-04 - mae: 0.0152 - mape: 15599.4912 - mse: 4.8210e-04 - val_loss: 6.7451e-04 - val_mae: 0.0184 - val_mape: 3.6475 - val_mse: 6.7451e-04
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 3.9761e-04 - mae: 0.0139 - mape: 2676.9336 - mse: 3.9761e-04 - val_loss: 6.9683e-04 - val_mae: 0.0190 - val_mape: 3.7465 - val_mse: 6.9683e-04
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 4.5866e-04 - mae: 0.0149 - mape: 2228.9319 - mse: 4.5866e-04 -

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
model_2: Processando ITUB4 | Combinação 4...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0673 - mae: 0.1781 - mape: 94928.2344 - mse: 0.0673 - val_loss: 9.9642e-04 - val_mae: 0.0238 - val_mape: 3.9488 - val_mse: 9.9642e-04
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 4.7404e-04 - mae: 0.0165 - mape: 20783.6758 - mse: 4.7404e-04 - val_loss: 6.4546e-04 - val_mae: 0.0182 - val_mape: 3.0386 - val_mse: 6.4546e-04
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.9802e-04 - mae: 0.0127 - mape: 53825.8555 - mse: 2.9802e-04 - val_loss: 9.0360e-04 - val_mae: 0.0231 - val_mape: 3.8768 - val_mse: 9.0360e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 3.2035e-04 - mae: 0.0133 - mape: 27209.4375 - mse: 3.2035e-04 - val_loss: 5.3944e-04 - val_mae: 0.0166 - val_mape: 2.7836 - val_mse: 5.3944e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 2.6950e-04 - mae: 0.0121 - mape: 39650.8867 - mse: 2

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


## 3. Processar o Backtest com Modelo salvo

In [ ]:
results, signals = process_stocks(stock_list, all_stock_data, 'model_2', 5) # num_combination = 5

Processando PETR4...


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
PETR4 processado!
Processando VALE3...


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
VALE3 processado!
Processando BBDC4...


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
BBDC4 processado!
Processando ITUB4...


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
ITUB4 processado!


In [9]:
plot_all_results(results, signals)

In [10]:
# Combinação 4: Stock Data + IT (IFR + MMS + MME)
final_results_df = calculate_final_results(results, signals)
final_results_df

,Ativo,Estratégia (Modelo 2) (R$),Buy & Hold (R$),Retorno Estratégia (%),Retorno Buy & Hold (%)
0,PETR4,R$ 9586.11,R$ 10739.39,-4.14%,7.39%
1,VALE3,R$ 14658.00,R$ 15489.48,46.58%,54.89%
2,BBDC4,R$ 10316.76,R$ 6553.20,3.17%,-34.47%
3,ITUB4,R$ 7430.84,R$ 6668.49,-25.69%,-33.32%


In [ ]:
# Combinação 5: Google News + Twitter + IFR + MMS
final_results_df = calculate_final_results(results, signals)
final_results_df

,Ativo,Estratégia (Modelo 2) (R$),Buy & Hold (R$),Retorno Estratégia (%),Retorno Buy & Hold (%)
0,PETR4,R$ 8472.75,R$ 10739.39,-15.27%,7.39%
1,VALE3,R$ 14991.93,R$ 15489.48,49.92%,54.89%
2,BBDC4,R$ 14635.71,R$ 6553.20,46.36%,-34.47%
3,ITUB4,R$ 7922.93,R$ 6668.49,-20.77%,-33.32%
